# Lab 02a: ISS decoding 

## Using starfish pixel spot decoder

Once the data is correctly loaded and processed, we are in the position of decoding the actual ISS spots. To this end, we will use again **starfish**, find the original tutorial [here](https://spacetx-starfish.readthedocs.io/en/mcai-api-additions/gallery/tutorials/pixelbased_decoding.html#sphx-glr-gallery-tutorials-pixelbased-decoding-py).

We prepared a bigger tile by cropping one of the stitched parts of the raw data. 

In [ ]:
import numpy as np
import os
from starfish import Experiment
from starfish.image import Filter
from starfish.spots import DetectPixels
from starfish.types import Features

In [ ]:
exp = Experiment.from_json(
    os.path.join('../data/in_situ_sequencing/SpaceTX', "experiment.json")
)
imgs = exp['fov_001'].get_image('primary')

Following the tutorial, first we will apply high and low pass filters.

In [ ]:
ghp = Filter.GaussianHighPass(sigma=4)
glp = Filter.GaussianLowPass(sigma=.5)
ghp.run(imgs, in_place=True)
glp.run(imgs, in_place=True)

We can already use some kind of decoding to detect the expressed genes. In this case, we will use a pixel spot decoder. This can yield suboptimal results in terms of detection, but we will still use if for simiplicity sake. There are other approaches for performing this such as [bardensr](https://github.com/jacksonloper/bardensr) or [ISTDECO](https://github.com/axanderssonuu/istdeco) that allow decoding with a better performance. 

There are some hyperparameters that need to be tuned, but the most important input to the function is the **codebook** that contain which combination of rounds and channels (barcode) is translated to a specific gene.

In [ ]:
psd = DetectPixels.PixelSpotDecoder(
    codebook=exp.codebook,
    metric='euclidean',             # distance metric to use for computing distance between a pixel vector and a codeword
    norm_order=2,                   # the L_n norm is taken of each pixel vector and codeword before computing the distance. this is n
    distance_threshold=0.5,         # minimum distance between a pixel vector and a codeword for it to be called as a gene
    magnitude_threshold=1.77e-5,    # discard any pixel vectors below this magnitude
    min_area=1,                     # do not call a 'spot' if it's area is below this threshold (measured in pixels)
    max_area=np.inf,                # do not call a 'spot' if it's area is above this threshold (measured in pixels)
)
initial_spot_intensities, prop_results = psd.run(imgs, n_processes=None)

These approaches usually yield too many false positives, so it is a good dea to threshold based on random codes included for this purpose.

In [ ]:
# filter spots that do not pass thresholds
spot_intensities = initial_spot_intensities.loc[initial_spot_intensities[Features.PASSES_THRESHOLDS]]

This is how the decoded intensity table looks like:

In [ ]:
spot_intensities

# Lab 02b: ISS decoding quality control

For this last step, we will use the TissUUmaps **spot inspector** plugin to visually assess the quality of the decoding. You will need to install TissUUmaps for this.

In [ ]:
from iss_utils import starfish2tmap

In [ ]:
csv_name = starfish2tmap.qc_csv(experiment=exp,
                     spot_intensities=spot_intensities,
                     output_path='../results/in_situ_sequencing')


In [ ]:
image_names = starfish2tmap.qc_images(imgs, exp, output_path='../results/in_situ_sequencing')

In [ ]:
image_names

In [ ]:
import tissuumaps.jupyter as tm

from urllib import request
os.makedirs(os.path.join(os.path.expanduser("~"), ".tissuumaps", "plugins"), exist_ok = True)
for ext in [".py",".js",".yml"]:
    request.urlretrieve(
        "https://tissuumaps.github.io/TissUUmaps/plugins/Spot_Inspector" + ext,
        os.path.join(os.path.expanduser("~"), ".tissuumaps", "plugins", 'Spot_Inspector' + ext)
    )

In [ ]:
tm.loaddata(images=image_names,csvFiles= csv_name, plugins=["Spot_Inspector"])